In [69]:
import pandas as pd
import numpy as np
import matplotlib as py
import re

In [47]:
reviews=pd.read_csv('/Users/anna/Downloads/reviews.csv.gz',compression='gzip').dropna()
listings=pd.read_csv('/Users/anna/Downloads/listings (1).csv',error_bad_lines=False)
neighborhoods=pd.read_csv('/Users/anna/Downloads/neighbourhoods.csv',error_bad_lines=False)

In [48]:
inactive=listings.loc[listings.availability_365==365]
inactive.set_index('id',inplace=True)
listings.set_index('id',inplace=True)
new_listings=listings.drop(inactive.index,axis=0)

In [55]:
new_listings.shape

(78074, 15)

In [50]:
reviews[:5]

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,13913,80770,2010-08-18,177109,Michael,My girlfriend and I hadn't known Alina before ...
1,13913,367568,2011-07-11,19835707,Mathias,Alina was a really good host. The flat is clea...
2,13913,529579,2011-09-13,1110304,Kristin,Alina is an amazing host. She made me feel rig...
3,13913,595481,2011-10-03,1216358,Camilla,"Alina's place is so nice, the room is big and ..."
4,13913,612947,2011-10-09,490840,Jorik,"Nice location in Islington area, good for shor..."


In [219]:
autopost=reviews[reviews.comments.str.contains('The host canceled')]
canceled=autopost.listing_id.value_counts()
num_cancel=canceled.to_frame(name='cancellations')
num_cancel[:5]

,cancellations
4461052,15
412058,15
6999328,14
10778211,14
23496564,14


In [67]:
df=new_listings.merge(num_cancel, how='left',left_index=True,right_index=True).fillna(0)

In [222]:
booked=365-df.availability_365
booked=booked.to_frame(name='days_booked')
df2=df.merge(booked, how='left',left_index=True,right_index=True)
df2[:5]

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,cancellations,days_booked
id,,,,,,,,,,,,,,,,,
11551,Stylish and bright London apartment in Zone 2,43039,Adriano,0,Lambeth,51.46225,-0.11732,Entire home/apt,88,2,183,2018-08-22,1.65,2,290,0,75
13913,Holiday London DB Room Let-on going,54730,Alina,0,Islington,51.56802,-0.11121,Private room,65,1,14,2018-06-17,0.13,3,362,1,3
15400,Bright Chelsea Apartment. Chelsea!,60302,Philippa,0,Kensington and Chelsea,51.48796,-0.16898,Entire home/apt,100,3,84,2019-04-22,0.74,1,155,0,210
17402,Superb 3-Bed/2 Bath & Wifi: Trendy W1,67564,Liz,0,Westminster,51.52098,-0.14002,Entire home/apt,300,3,40,2019-04-20,0.40,13,230,0,135
24328,Battersea bedroom & office w parking shared ga...,41759,Joe,0,Wandsworth,51.47298,-0.16376,Entire home/apt,175,30,92,2016-09-07,0.89,1,332,2,33


In [221]:
autopost.reset_index()[:5]

,index,listing_id,id,date,reviewer_id,reviewer_name,comments
0,12,13913,262179591,2018-05-08,99110277,Philippe,The host canceled this reservation 53 days bef...
1,136,24328,221644,2011-04-10,315279,Craig,The host canceled my reservation 38 days befor...
2,224,24328,98123248,2016-08-29,29270232,Keira,The host canceled this reservation 58 days bef...
3,361,25123,212211101,2017-11-16,156529186,Krishnamoortee,The host canceled this reservation 35 days bef...
4,431,26482,137001,2010-11-10,278167,Benjamin,The host canceled my reservation 52 days befor...


**QUESTION**

**For the next step, I want to create a separate column for the number of days left before the guest's arrival when the cancellation was made. BUT I'm having trouble doing this. To extract the number of days, i'm using re.findall(). I'm having a hard time compiling this in a list/series.**

In [216]:
list_days=[]

for row in autopost.comments:
    x=(re.findall('\d+', row))
    list_days+=x
    
print(len(list_days))

#there are supposed to be 21,341 rows.. which means there are null values(i'd have to figure why there are null values 
#later on as well). since the null values are being ignored in this code, i try to deal with them in the codes below but 
#it's not working either



19842


In [212]:
# the problem arises when i try to deal with the null values. i try to do if statements but it's not working and i'm not
#sure why? my thought process for this code was that along with the number of days, i want there to be a corresponding
#index number so i can successfully merge them to the dataframe above

#shouldn't 'index=list(autopost.comments).index(string)' give me an index number that i can compile in my list 'ind'? 
days_prior=[]
ind=[]
for string in autopost.comments:
    days=re.findall('\d+', string)
    index=list(autopost.comments).index(string)
    if days>=0:
        days_prior+=days
        ind+=index
    else:
        days_prior+=0
        ind+=index
    
    
    
    
print(days_prior)
print(ind)



TypeError: 'int' object is not iterable

In [224]:
#for this one, i'm not even trying to deal with the index anymore. i just want to make sure the null values are 
#included but it's separating them by digit. isn't 'days' a list in this code? so shouldn't 'x in days' be the 
#object in the list? and not the separate digits?


list=[]

for string in autopost.comments:
    days=(re.findall('\d+', string))
    for x in days:
        if x>=0:
            list+=x
        else:
            list+=0
 
    
print(list)[:20]

['5', '3', '3', '8', '5', '8', '3', '5', '5', '2', '7', '3', '8', '7', '1', '5', '7', '1', '3', '3']
